In [ ]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_openai import ChatOpenAI
api_key = 'sk-xxxx'  # 我的


deepseek_r1 = ChatOpenAI(model="deepseek-r1", temperature=0, api_key=api_key, 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

best_model = ChatOpenAI(model="deepseek-v3", temperature=0, api_key=api_key, 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

deepseek_v3 = ChatOpenAI(model="deepseek-v3", temperature=0, api_key=api_key, 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

kimi_model = ChatOpenAI(model="Moonshot-Kimi-K2-Instruct", temperature=0, api_key=api_key, 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")


base_model = ChatTongyi(
    model="qwen3-235b-a22b",
    api_key=api_key,temperature=0
)
tool_agent = ChatTongyi(
    model="qwen3-235b-a22b",
    api_key=api_key,temperature=0
)

In [2]:
industry_name = '中国智能服务机器人产业'
industry_input = industry_name
task2 = f"""
完成 {industry_input} 的研报。
行业/子行业研报应能够聚合行业发展相关数据（协会年报、企业财报等），
输出行业生命周期与结构解读（如集中度、产业链上下游分析）；
融合趋势分析与外部变量预测能力（如政策影响、技术演进），支持3年以上的行业情景模拟；
提供行业进入与退出策略建议，支持关键变量（如上游原材料价格）敏感性分析；
自动生成图表辅助说明行业规模变动、竞争格局等核心要素。
"""

# 1. 获取行业的top的公司信息

In [ ]:
## 判断最相关的行业代码
# find_industry_top_stock_code = f"""
#     你是一个金融专家，请你分析以下行业最具有代表性的公司，行业是：{industry_input}。
#     请你返回这个行业最具代表性的公司的港股代码，请只返回港股代码，不要有其他内容。
# """  '08083.HK'

# find_industry_top_stock_code = f"""
#     你是一个金融专家，请你分析以下行业最具有代表性的公司，行业是：{industry_input}。
#     请你返回这个行业最具代表性的公司的港股代码，请只返回港股代码，不要有其他内容。
# """  '08083.HK'

find_industry_top_stock_code = f"""
    你是一个金融专家，请你分析以下行业最具有代表性的公司，行业是：{industry_input}。
    请只返回该公司名及其A股代码，不要有其他内容。
""" 

industry_top_stock_response = best_model.invoke(find_industry_top_stock_code)
industry_top_stock = industry_top_stock_response.content.strip()

# 返回结果为  '科沃斯（603486）'

In [ ]:
def judge_stock_type(input_stock_code: str):
    """
    调用该函数，从混合着股票代码和公司信息的输入判断股票是A股的还是港股的，并提取出准确的股票代码。
    Args:
        input_stock_code (str): 输入的股票代码和公司信息
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, str]): 包含股票类型（A股或港股）和对应的股票代码
            data_type (str): 数据类型
            data_desc (str): 数据描述
    例如：
    """
    prompt = """你是一个股票研究专家，请判断这个股票代码是A股还是港股，并返回股票代码。对于A股，返回6位纯数字的股票代码；对于港股，返回4位纯数字的股票代码。
    股票代码和公司信息是：{stock_code}
    请只返回股票代码，不要包含其他任何信息。你输出的股票代码为："""
    response = base_model.invoke(prompt.format(stock_code=input_stock_code),enable_thinking=False)
    stock_code = response.content.strip()
    if len(stock_code) == 6:
        stock_type = 'A股'
    else:
        stock_type = '港股'
    return stock_type, stock_code
# 1. 提取股票类别和股票代码
stock_type, stock_code = judge_stock_type(industry_top_stock)

# 返回结果为 ('A股', '603486')

In [7]:
import akshare as ak
import pandas as pd
from get_capital_structure import get_capital_structure_cn_ths, get_capital_structure_hk_ths
from get_cn_control import get_cn_control
from get_company_field_compare import get_cn_company_field_compare, get_hk_company_field_compare
from get_company_finance_summary import get_company_finance_summary_cn,get_company_finance_summary_hk
from get_company_intro import get_company_profile_ths_cn, get_company_profile_ths_hk, get_cn_company_profile_ak, get_hk_company_profile_ak
from get_financial_data_annual import download_cn_financial_data, download_hk_financial_data
from get_rating_info import get_hk_rating_info, get_cn_rating_info
from get_stock_info import get_cn_stock_info, get_hk_stock_info
from get_worth_predict import get_cn_worth_predict, get_hk_worth_predict
from datetime import datetime, timedelta
import time
import requests
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from typing import Dict, Any
from typing_extensions import Annotated, TypedDict
import json
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from typing import Dict, Optional, Literal
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
import ast
import pickle

from typing import List

from langchain_core.tools import InjectedToolArg, tool
from typing_extensions import Annotated
import time
from time import sleep
# 支持matplotlib输出中文
import matplotlib.pyplot as plt
import chineseize_matplotlib
plt.rcParams['axes.unicode_minus'] = False # 解决负号'-'显示为方块的问题

In [8]:
################### 6. 获取行业对比信息

def extract_field_compare(stock_type, stock_code):
    """
    提取行业对比分析数据
    Args:
        stock_type (str): 股票类型
        stock_code (str): 股票代码
    Returns:
        Dict[str, Any]: 包含以下键的字典：
            data_value (Dict[str, dict]): 行业对比分析数据，包含目标公司和同行业其他公司的对比数据
            data_type (str): 数据类型
            data_desc (str): 数据内容描述，说明返回的数据包含哪些行业对比分析数据
    """
    if stock_type == '港股':
        hk_code = ''.join(filter(str.isdigit, hk_code))  # 只保留数字
        if len(hk_code) > 4:
            hk_code = hk_code[-4:]
        hk_code = '0' + hk_code  # 确保是5位数字
        hk_code = 'HK' + hk_code
        field_compare = get_hk_company_field_compare(hk_code)  # 港股包括
    else:
        a_code = stock_code
        field_compare = get_cn_company_field_compare(a_code)

    issue_list = []
    title_list = []
    ps_list = []
    dataframe_list = []
    for item in field_compare:
        issue_list.append(item['issue'])
        title_list.append(item['title'])
        ps_list.append(item['ps'])
        dataframe_list.append(item['table'])
    return {
        'data_value':{
            "issue": issue_list,
            "title": title_list,
            "ps": ps_list,
            "table": dataframe_list
        },
        'data_type': 'dict_special_list',
        'data_desc': '行业对比分析数据，包含目标公司和同行业其他公司的对比数据，是一个list，包括issue，title，ps和数据table，table是原始数据dataframe'
    }


In [9]:
# 信息获取后，数据存储到dict里面
collected_data_value = {}
collected_data_type = {}
collected_data_desc = {}

In [10]:
def save_extracted_info(data_value,
                        data_type,
                        data_desc):
    global collected_data_value, collected_data_type, collected_data_desc
    if data_type=='dict': # 普通的dict直接放进去就可以了
        collected_data_value.update(data_value)
        # 注意提取出data_value里面的key作为后面的数据类型和描述的key
        for key in data_value.keys():
            collected_data_type[key] = 'str'
            collected_data_desc[key] = str(key)
    elif data_type=='dict_dataframe':
        collected_data_value.update(data_value)
        # 注意提取出data_value里面的key作为后面的数据类型和描述的key
        for key in data_value.keys():
            collected_data_type[key] = 'dataframe'
            collected_data_desc[key] = data_desc
    elif data_type=='dict_special_list': # 最复杂的一种，这个的data_value是一个list
        for i in range(len(data_value['issue'])):
            issue = data_value['issue'][i]
            title = data_value['title'][i]
            ps = data_value['ps'][i]
            dataframe = data_value['table'][i]
            description = f"{issue} - {title} - ({ps})"
            if '行业分类' in title:
                key = str(issue) + '行业财务数据'
            else:
                if '价值' in title or '表现' in title:
                    key = str(issue) + '行业价值表现数据'
                else:
                    key = str(issue) + '行业财务经营数据'
            collected_data_value[key] = dataframe
            collected_data_type[key] = 'dataframe'
            collected_data_desc[key] = description


import pickle

def save_collected_data_to_local():
    with open('field_collected_data_value.pkl', 'wb') as f:
        pickle.dump(collected_data_value, f)
    with open('field_collected_data_type.pkl', 'wb') as f:
        pickle.dump(collected_data_type, f)
    with open('field_collected_data_desc.pkl', 'wb') as f:
        pickle.dump(collected_data_desc, f)

    print("数据已成功保存到本地。")

def load_collected_data_from_local():
    global collected_data_value, collected_data_type, collected_data_desc
    try:
        with open('field_collected_data_value.pkl', 'rb') as f:
            collected_data_value = pickle.load(f)
        with open('field_collected_data_type.pkl', 'rb') as f:
            collected_data_type = pickle.load(f)
        with open('field_collected_data_desc.pkl', 'rb') as f:
            collected_data_desc = pickle.load(f)
        print("数据已从本地加载。")
    except FileNotFoundError:
        print("未找到本地数据文件，请先运行保存操作。")


# 7. 提取行业对比分析数据
field_compare_data = extract_field_compare(stock_type, stock_code)
save_extracted_info(
    data_value=field_compare_data['data_value'],
    data_type=field_compare_data['data_type'],
    data_desc=field_compare_data['data_desc']
)
save_collected_data_to_local()


数据已成功保存到本地。


# 2. 获取研报信息，取东方财富的

## 2.1 获取行业代码

In [3]:
# 东方财富的  行业代码:行业名

# 先从industry_dict中获取最相似的行业，获取到对应的key

import requests
from bs4 import BeautifulSoup
src_mapping = """<div id="hymore" class="select-box" style="display: none;"><ul><li><b>B</b><a target="_self" data-bkval="546">玻璃玻纤</a></li><li><a target="_self" data-bkval="474">保险</a></li><li><a target="_self" data-bkval="1036">半导体</a></li><li><a target="_self" data-bkval="733">包装材料</a></li><li><b>C</b><a target="_self" data-bkval="1017">采掘行业</a></li><li><a target="_self" data-bkval="729">船舶制造</a></li><li><b>D</b><a target="_self" data-bkval="459">电子元件</a></li><li><a target="_self" data-bkval="457">电网设备</a></li><li><a target="_self" data-bkval="428">电力行业</a></li><li><a target="_self" data-bkval="1039">电子化学品</a></li><li><a target="_self" data-bkval="1034">电源设备</a></li><li><a target="_self" data-bkval="1033">电池</a></li><li><a target="_self" data-bkval="1030">电机</a></li><li><a target="_self" data-bkval="738">多元金融</a></li><li><b>F</b><a target="_self" data-bkval="451">房地产开发</a></li><li><a target="_self" data-bkval="436">纺织服装</a></li><li><a target="_self" data-bkval="1045">房地产服务</a></li><li><a target="_self" data-bkval="1032">风电设备</a></li><li><a target="_self" data-bkval="1020">非金属材料</a></li><li><b>G</b><a target="_self" data-bkval="479">钢铁行业</a></li><li><a target="_self" data-bkval="427">公用事业</a></li><li><a target="_self" data-bkval="425">工程建设</a></li><li><a target="_self" data-bkval="1038">光学光电子</a></li><li><a target="_self" data-bkval="1031">光伏设备</a></li><li><a target="_self" data-bkval="739">工程机械</a></li><li><a target="_self" data-bkval="732">贵金属</a></li><li><a target="_self" data-bkval="726">工程咨询服务</a></li><li><b>H</b><a target="_self" data-bkval="538">化学制品</a></li><li><a target="_self" data-bkval="480">航天航空</a></li><li><a target="_self" data-bkval="471">化纤行业</a></li><li><a target="_self" data-bkval="465">化学制药</a></li><li><a target="_self" data-bkval="450">航运港口</a></li><li><a target="_self" data-bkval="447">互联网服务</a></li><li><a target="_self" data-bkval="420">航空机场</a></li><li><a target="_self" data-bkval="1019">化学原料</a></li><li><a target="_self" data-bkval="731">化肥行业</a></li><li><a target="_self" data-bkval="728">环保行业</a></li><li><b>J</b><a target="_self" data-bkval="456">家电行业</a></li><li><a target="_self" data-bkval="440">家用轻工</a></li><li><a target="_self" data-bkval="429">交运设备</a></li><li><a target="_self" data-bkval="740">教育</a></li><li><a target="_self" data-bkval="735">计算机设备</a></li><li><b>L</b><a target="_self" data-bkval="485">旅游酒店</a></li><li><b>M</b><a target="_self" data-bkval="484">贸易行业</a></li><li><a target="_self" data-bkval="437">煤炭行业</a></li><li><a target="_self" data-bkval="1035">美容护理</a></li><li><b>N</b><a target="_self" data-bkval="477">酿酒行业</a></li><li><a target="_self" data-bkval="433">农牧饲渔</a></li><li><a target="_self" data-bkval="1015">能源金属</a></li><li><a target="_self" data-bkval="730">农药兽药</a></li><li><b>Q</b><a target="_self" data-bkval="481">汽车零部件</a></li><li><a target="_self" data-bkval="1029">汽车整车</a></li><li><a target="_self" data-bkval="1016">汽车服务</a></li><li><b>R</b><a target="_self" data-bkval="1028">燃气</a></li><li><a target="_self" data-bkval="737">软件开发</a></li><li><b>S</b><a target="_self" data-bkval="482">商业百货</a></li><li><a target="_self" data-bkval="464">石油行业</a></li><li><a target="_self" data-bkval="454">塑料制品</a></li><li><a target="_self" data-bkval="438">食品饮料</a></li><li><a target="_self" data-bkval="424">水泥建材</a></li><li><a target="_self" data-bkval="1044">生物制品</a></li><li><b>T</b><a target="_self" data-bkval="545">通用设备</a></li><li><a target="_self" data-bkval="448">通信设备</a></li><li><a target="_self" data-bkval="421">铁路公路</a></li><li><a target="_self" data-bkval="736">通信服务</a></li><li><b>W</b><a target="_self" data-bkval="486">文化传媒</a></li><li><a target="_self" data-bkval="422">物流行业</a></li><li><b>X</b><a target="_self" data-bkval="1037">消费电子</a></li><li><a target="_self" data-bkval="1027">小金属</a></li><li><a target="_self" data-bkval="1018">橡胶制品</a></li><li><b>Y</b><a target="_self" data-bkval="478">有色金属</a></li><li><a target="_self" data-bkval="475">银行</a></li><li><a target="_self" data-bkval="458">仪器仪表</a></li><li><a target="_self" data-bkval="1046">游戏</a></li><li><a target="_self" data-bkval="1042">医药商业</a></li><li><a target="_self" data-bkval="1041">医疗器械</a></li><li><a target="_self" data-bkval="727">医疗服务</a></li><li><b>Z</b><a target="_self" data-bkval="539">综合行业</a></li><li><a target="_self" data-bkval="476">装修建材</a></li><li><a target="_self" data-bkval="473">证券</a></li><li><a target="_self" data-bkval="470">造纸印刷</a></li><li><a target="_self" data-bkval="1043">专业服务</a></li><li><a target="_self" data-bkval="1040">中药</a></li><li><a target="_self" data-bkval="910">专用设备</a></li><li><a target="_self" data-bkval="734">珠宝首饰</a></li><li><a target="_self" data-bkval="725">装修装饰</a></li><ul></ul></ul></div>"""
soup = BeautifulSoup(src_mapping, 'html.parser')

industry_dict = {}

for a_tag in soup.select('#hymore a[data-bkval]'):
    bid = a_tag.get('data-bkval')
    name = a_tag.text.strip()
    if bid and name:
        industry_dict[bid] = name

print(industry_dict)

## {'546': '玻璃玻纤', '474': '保险', '1036': '半导体'}等

{'546': '玻璃玻纤', '474': '保险', '1036': '半导体', '733': '包装材料', '1017': '采掘行业', '729': '船舶制造', '459': '电子元件', '457': '电网设备', '428': '电力行业', '1039': '电子化学品', '1034': '电源设备', '1033': '电池', '1030': '电机', '738': '多元金融', '451': '房地产开发', '436': '纺织服装', '1045': '房地产服务', '1032': '风电设备', '1020': '非金属材料', '479': '钢铁行业', '427': '公用事业', '425': '工程建设', '1038': '光学光电子', '1031': '光伏设备', '739': '工程机械', '732': '贵金属', '726': '工程咨询服务', '538': '化学制品', '480': '航天航空', '471': '化纤行业', '465': '化学制药', '450': '航运港口', '447': '互联网服务', '420': '航空机场', '1019': '化学原料', '731': '化肥行业', '728': '环保行业', '456': '家电行业', '440': '家用轻工', '429': '交运设备', '740': '教育', '735': '计算机设备', '485': '旅游酒店', '484': '贸易行业', '437': '煤炭行业', '1035': '美容护理', '477': '酿酒行业', '433': '农牧饲渔', '1015': '能源金属', '730': '农药兽药', '481': '汽车零部件', '1029': '汽车整车', '1016': '汽车服务', '1028': '燃气', '737': '软件开发', '482': '商业百货', '464': '石油行业', '454': '塑料制品', '438': '食品饮料', '424': '水泥建材', '1044': '生物制品', '545': '通用设备', '448': '通信设备', '421': '铁路公路', '736': '通信服务', '486': '

In [6]:
## 判断最相关的行业代码
find_most_similar_industry = f"""
    你是一个金融行业的专家，你需要从 行业代码:行业名 的字典中，找到与输入的行业描述最相似的3个行业及其代码。
    行业代码:行业名 的字典为
    -----
    {industry_dict}
    -----

    请从上面的行业代码:行业名 的字典中，仔细分析，找到与 {industry_input} 最相似的3个行业及其代码，并输出为json格式，不要解释。
"""

similar_industry_response = kimi_model.invoke(find_most_similar_industry)
similar_industry = similar_industry_response.content.strip()

## 判断最相关的行业代码
find_most_similar_industry_think_template = """
    你是一个金融行业的专家，你需要从 行业代码:行业名 的字典中，找到与输入的行业描述最相似的1个行业及其代码。
    行业代码:行业名 的字典为
    -----
    {industry_dict}
    -----

    请从上面的行业代码:行业名 的字典中，仔细分析，找到与 {industry_input} 最相似的1个行业及其代码。
    请仔细思考，不要停留在字面意思，需要从行业的本质、发展趋势、市场需求等方面进行综合分析。
    输出为json格式，不要解释。
"""
find_most_similar_industry_think = find_most_similar_industry_think_template.format(
    industry_dict=similar_industry_response, industry_input=industry_input)

most_similar_industry_response = best_model.invoke(find_most_similar_industry_think)
most_similar_industry = most_similar_industry_response.content.strip()



# 提取行业代码，只需解析出第一个数字即可
import re
def extract_industry_code(industry_str):
    match = re.search(r'\d+', industry_str)
    return match.group(0) if match else None
industry_code = extract_industry_code(most_similar_industry)
# print最相似的行业名和对应代码 可以从industry_dict中获取
industry_name = industry_dict.get(industry_code, "未知行业")
print(f"最相似的行业代码: {industry_code}, 行业名: {industry_name}")

industry_code = int(industry_code)
# 最相似的行业代码: 1037, 行业名: 消费电子

最相似的行业代码: 1037, 行业名: 消费电子


## 2.2 获取行业报告列表，第1页（每页50个报告）的标题和详细内容的网址

In [6]:
# 原始的获取研报的api，慢一点sleep多一点
import requests
import json
import time
from datetime import datetime, timedelta

def get_dfcf_research_report(industry_code, page=1, years_ago=2):
    """
    获取东方财富网的行业研报数据。
    :param industry_code: 行业代码
    :param page: 页码，默认为1
    """
    # API URL
    url = "https://reportapi.eastmoney.com/report/list"

    # 生成当前时间戳（毫秒级）
    timestamp = int(time.time() * 1000)
    # 当天年月日
    current_date = datetime.now().strftime('%Y-%m-%d')

    # 将日期字符串转为 datetime 对象
    date_obj = datetime.strptime(current_date, '%Y-%m-%d')

    two_years_ago = date_obj.replace(year=date_obj.year - years_ago)

    # 格式化回字符串
    two_years_ago_str = two_years_ago.strftime('%Y-%m-%d')
    # 请求参数
    params = {
        'industryCode': str(industry_code),
        'pageSize': 50, # 每页多少条
        'industry': '*',
        'rating': '*',
        'ratingChange': '*',
        'beginTime': two_years_ago_str, # 2年前的
        'endTime': current_date,  # 使用系统提供的时间：2025年6月27日
        'pageNo': page,
        'fields': '',
        'qType': 1,
        'orgCode': '',
        'rcode': '',
        'p': page,
        'pageNum': page,
        'pageNumber': page,
        '_': timestamp,  # 使用动态生成的时间戳
    }

    # 设置请求头（模拟浏览器访问，防止反爬虫机制）
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0 Safari/537.36'
    }

    # 发送GET请求
    response = requests.get(url, params=params, headers=headers)

    # 检查响应状态码
    if response.status_code == 200:
        try:
            # 移除回调函数并解析JSON数据
            json_data = json.loads(response.text.replace('datatable3015936(', '').rstrip(')'))
            #print(json.dumps(json_data, ensure_ascii=False, indent=4))  # 打印格式化后的JSON数据
        except json.JSONDecodeError as e:
            print(f"Failed to decode JSON: {e}")
    else:
        print(f"Failed to retrieve data: HTTP Status Code {response.status_code}")

    return json_data

In [7]:
# 获取第一页的研报数据
from time import sleep
import pandas as pd
idx_list = []
title_list = []
infocede_list = []
url_list = []
report_date_list = []

max_page = 1  # 最大获取研报页数
idx_cnt = 0
for page in range(1,max_page+1):
    print(f"正在获取第 {page} 页的研报数据...")
    sleep(20)  # 东方财富网的反爬虫机制，适当延时
    dfcs_top50_research_url = get_dfcf_research_report(industry_code, page=page, years_ago=2)
    with open(f"dfcf_research_report_list_for_{industry_code}_page_{page}.json", 'w', encoding='utf-8') as f:
        json.dump(dfcs_top50_research_url, f, ensure_ascii=False, indent=4)
    for i in range(len(dfcs_top50_research_url['data'])):
        idx_list.append(idx_cnt)
        title_list.append(dfcs_top50_research_url['data'][i]['title'])
        infocede_list.append(dfcs_top50_research_url['data'][i]['infoCode'])
        url_list.append('https://data.eastmoney.com/report/zw_industry.jshtml?infocode=' + dfcs_top50_research_url['data'][i]['infoCode'])
        report_date = dfcs_top50_research_url['data'][i]['publishDate']
        report_date = pd.to_datetime(report_date).strftime('%Y-%m-%d')
        report_date_list.append(report_date)
        idx_cnt += 1

# 存储为csv文件
import pandas as pd
dfcs_research_df = pd.DataFrame({
    'idx': idx_list,
    'title': title_list,
    'report_date': report_date_list,
    'infocode': infocede_list,
    'url': url_list
})
dfcs_research_df.to_csv(f'dfcs_research_report_list_for_{industry_code}.csv', index=False, encoding='utf-8')

正在获取第 1 页的研报数据...


In [8]:
# 获取研报内容
def get_dfcf_research_report_content(url):
    """
    获取东方财富网的行业研报内容。
    :param url: 研报的URL
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve report content: HTTP Status Code {response.status_code}")
        return None


# 解析研报内容
from bs4 import BeautifulSoup
# 找到 div ctx-content，里面是研报内容
def parse_dfcf_research_report_content(html_content):
    """
    解析东方财富网的行业研报内容。
    :param html_content: 研报的HTML内容
    :return: 研报的文本内容
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    content_div = soup.find('div', class_='ctx-content')
    
    if content_div:
        return content_div.get_text(strip=True)
    else:
        print("Content div not found.")
        return None


url_content = []
for i in range(len(dfcs_research_df)):
    url = dfcs_research_df['url'][i]
    print(f"正在获取第 {i+1} 条研报内容，链接为：{url}")
    sleep(20)  # 东方财富网的反爬虫机制，适当延时
    report_content = get_dfcf_research_report_content(url_list[i])
    if report_content:
        # 解析第一个研报的内容
        parsed_content = parse_dfcf_research_report_content(report_content) 
        if parsed_content:
            # 将解析后的内容添加到列表中
            url_content.append(parsed_content)
        else:
            url_content.append("空")
    else:
        url_content.append("空")

# 保存研报内容到CSV文件
dfcs_research_df['content'] = url_content
dfcs_research_df.to_csv(f'dfcs_research_report_content_for_{industry_code}.csv', index=False, encoding='utf-8')

正在获取第 1 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507231714155260
正在获取第 2 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507181711146464
正在获取第 3 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507111706966574
正在获取第 4 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507091705618510
正在获取第 5 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507081705064841
正在获取第 6 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507081704978340
正在获取第 7 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507071704408487
正在获取第 8 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202507031702258128
正在获取第 9 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP202506291699838375
正在获取第 10 条研报内容，链接为：https://data.eastmoney.com/report/zw_industry.jshtml?infocode=AP20250629

# 3. 加载数据，构建数据chunk

In [13]:
import pickle
import pandas as pd
# 信息获取后，数据存储到dict里面
collected_data_value = {}
collected_data_type = {}
collected_data_desc = {}

def load_collected_data_from_local():
    global collected_data_value, collected_data_type, collected_data_desc
    try:
        with open('field_collected_data_value.pkl', 'rb') as f:
            collected_data_value = pickle.load(f)
        with open('field_collected_data_type.pkl', 'rb') as f:
            collected_data_type = pickle.load(f)
        with open('field_collected_data_desc.pkl', 'rb') as f:
            collected_data_desc = pickle.load(f)
        print("数据已从本地加载。")
    except FileNotFoundError:
        print("未找到本地数据文件，请先运行保存操作。")

load_collected_data_from_local()

数据已从本地加载。


In [14]:
# 研报数据加载
df_research_data = pd.read_csv(f'dfcs_research_report_content_for_{industry_code}.csv', encoding='utf-8')

In [15]:
# 构建数据chunk
data_chunk = []
data_chunk_disc = []
for key, value in collected_data_value.items():
    if isinstance(value, pd.DataFrame):
        # 转化为字符串
        value_str = value.to_csv(index=False, encoding='utf-8')
        data_chunk.append(value_str)
        data_chunk_disc.append('dataframe-'+collected_data_desc[key])
    else:
        data_chunk.append(value)
        data_chunk_disc.append(collected_data_desc[key])

# df_research_data里面content放到value，title放到desc
for i in range(len(df_research_data)):
    content = df_research_data.loc[i, 'content']
    title = df_research_data.loc[i, 'title']
    data_chunk.append(content)
    data_chunk_disc.append('行业研究报告-'+title)

In [16]:
# 构建数据块描述的字符串
all_data_chunks = []
def construct_collected_data_chunks(collected_data_value,collected_data_desc):
    global all_data_chunks
    data_chunks_disc_list = []
    idx = 1
    for i in range(len(collected_data_value)):
        desc = collected_data_desc[i]
        value = collected_data_value[i]
        discription = f"数据块{idx}: {desc}\n"
        data_chunks_disc_list.append(discription)
        # 如果是DataFrame类型的数据，转换为字符串
        if isinstance(value, pd.DataFrame):
            table_intro = f'{desc}'
            table_str = value.to_csv(sep='|', index=False, header=True)
            result_str = f"{table_intro}\n{table_str}"
        else:
            # 如果是其他类型的数据，直接转换为字符串
            result_str = f"{desc}\n\n{str(value)}"
        all_data_chunks.append(result_str)  # 添加数据块内容
        idx += 1
    return "\n".join(data_chunks_disc_list)

data_chunks_disc = construct_collected_data_chunks(data_chunk, data_chunk_disc)

In [17]:

# 整体的trace的log
trace_log = "trace2.log"
print_log = "print2.log"

def write_print_to_log(text,path='print2.log'):
    """
    将文本写入日志文件
    :param text: 要写入的文本
    :param path: 日志文件路径
    """
    with open(path, 'a', encoding='utf-8') as f:
        f.write(text + '\n')
        print(text)  # 同时输出到控制台


# 4.RAG报告撰写，同任务一

In [18]:
sub_task_plan_template = """
你是一个专业的金融研究员，需要撰写一篇行业金融研究报告。具体要研究的行业和任务如下：

任务说明：
------------
{query}
------------

请你根据这个研究任务，按照逻辑顺序拆解为多个子任务。每个子任务应具备以下特征：
- 可以独立完成
- 按照写作或研究的先后顺序排列
- 数量不超过 10 个子任务

输出格式要求如下：
- 使用如下格式：
  #1# 子任务说明
  #2# 子任务说明
  ...
- 不使用任何 Markdown 格式
- 不添加任何解释性内容

请开始输出子任务计划。
"""
sub_task_plan_prompt = sub_task_plan_template.format(query=task2)

sub_task_response = deepseek_v3.invoke(sub_task_plan_prompt)


import re
def generate_subtasks(llm_response):
    # 提取子任务
    subtasks = re.findall(r'#(\d+)#\s*(.+)', llm_response.strip())
    # 按顺序排序并去重
    subtasks = sorted(subtasks, key=lambda x: int(x[0]))
    unique_tasks = {}
    for idx, task in subtasks:
        if task not in unique_tasks:
            unique_tasks[task] = None
    # 返回任务列表
    return list(unique_tasks.keys())

task_list = generate_subtasks(sub_task_response.content)
task_list

['收集中国智能服务机器人产业相关数据，包括协会年报、企业财报、市场规模等  ',
 '分析行业生命周期阶段，判断当前行业处于导入期、成长期、成熟期或衰退期  ',
 '研究行业集中度和竞争格局，包括市场份额、主要企业及竞争策略  ',
 '进行产业链上下游分析，包括上游原材料供应、中游制造、下游应用领域  ',
 '评估外部变量对行业的影响，如政策法规、技术演进、宏观经济环境等  ',
 '构建行业情景模拟模型，预测未来3-5年行业发展趋势  ',
 '提供行业进入与退出策略建议，包括投资机会和风险规避  ',
 '进行关键变量敏感性分析，如上游原材料价格波动对行业的影响  ',
 '生成图表辅助说明行业规模变动、竞争格局等核心要素  ',
 '整合所有分析结果，撰写完整行业研究报告并审核定稿']

In [25]:
# 获取最相关的数据chunk块
find_most_relavent_chunks_template = """
    你是一个专业的金融研究员，需要对关于 {industry_input} 行业的问题给出非常专业精准的回答，因此你需要先从候选的数据池中选择出最相关的数据块以提供参考信息。
    该问题为：
    -----
    {task}
    -----

    为了回答该问题，你需要从以下数据池中选择最相关的数据块，获取信息作为参考：
    -----
    {data_chunks}
    -----

    请你筛选出回答该问题最有可能用到的数据块编号，最多选择6个数据块，最少选择一个数据块，每个数据块的编号之间用逗号分隔，直接输出编号，不要添加任何其他内容，例如 0,1
    不要解释、分析或者输出任何其他内容，只需要输出数据块标号，你判断需要使用的数据块标号是：
"""

# 用于生成报告正文内容的模板
subtask_solution_prompt_template = """
你是一个专业的金融研究员，正在撰写关于 {industry_input} 的行业研究报告。

当前章节需要涵盖的内容是：
-----
{task}
-----

你需要参考以下数据和信息：
-----
{used_data_chunk}
-----

请根据以上信息，撰写该章节的正式报告内容。要求如下：

1. 表述专业、逻辑清晰、内容翔实，适合用于正式研究报告；
2. 若涉及数据，应保证数据和参考信息里面一致，不能编造；
3. 若涉及表格，应使用Markdown格式输出表格，表格内容要清晰、易读，可以只呈现关键数据，例如按照年份进行采样或者汇总输出，但是表格内容要完整不能出现省略号；
4. 若涉及分析，应给出结论性判断（如趋势、对比、风险、机会等）；
5. 不要写“我将如何分析”，而是直接输出分析结论；
6. 不要写“步骤一、步骤二”；
7. 使用中文，语句通顺，格式规范。

请直接以正式报告内容的形式输出该章节内容。
"""

In [26]:
# import sleep相关
from time import sleep
task_solution_list = []
for i in range(len(task_list)): # 依次完成任务
    task = task_list[i]  # 当前任务
    write_print_to_log(f"\n\n--------------------开始处理任务 {i+1}: {task} --------------------\n")
    retry = 0
    finish_flag = 0
    while retry<3 and finish_flag<=0:
        # 搜集和任务相关的信息，可以取出最相关的6个chunk
        find_most_relavent_chunks_prompt = find_most_relavent_chunks_template.format(
            industry_input=industry_input,  # 输入的股票代码
            task=task,  # 当前的任务
            data_chunks=data_chunks_disc  # 数据块描述字符串
        )
        find_chunk_response = deepseek_v3.invoke(find_most_relavent_chunks_prompt)
        sleep(1)
        most_relevant_chunks = find_chunk_response.content.strip()  # 最相关的数据块编号
        # 解析chunk编号
        most_relevant_chunks_list = most_relevant_chunks.split(',')  # 分割成列表
        # 检查数据格式是否准确
        most_relevant_chunks_list = [chunk.strip() for chunk in most_relevant_chunks_list if chunk.strip().isdigit()]
        if len(most_relevant_chunks_list) == 0:
            write_print_to_log(f"没有找到相关的数据块，任务 {i+1} 无法完成")
            most_relevant_chunks_list = [0]
        write_print_to_log(f"最相关的数据块编号: {most_relevant_chunks_list}")
        # trace_log输出
        with open(trace_log, 'a', encoding='utf-8') as f:
            f.write(f"Most relevant chunks: {most_relevant_chunks}\n")
        finish_flag = 1 # 有数据了
        # 构建数据块字符串
        used_data_chunk = []
        for k in range(len(most_relevant_chunks_list)):
            chunk_index = int(most_relevant_chunks_list[k])
            if chunk_index < len(all_data_chunks):
                used_data_chunk.append(all_data_chunks[chunk_index])
        
        used_data_chunk_str = '\n\n'.join(used_data_chunk)

        # 生成任务解决方案
        subtask_solution_prompt = subtask_solution_prompt_template.format(
            industry_input=industry_input,  # 输入的行业
            task=task,  # 当前的任务
            used_data_chunk=used_data_chunk_str  # 使用的数据块字符串
        )

        subtask_solution_response = deepseek_r1.invoke(subtask_solution_prompt)
        sleep(1)
        subtask_solution = subtask_solution_response.content
        task_solution_list.append(subtask_solution)  # 存储任务解决方案
        # trace_log输出
        with open(trace_log, 'a', encoding='utf-8') as f:
            f.write(f"Subtask solution: {subtask_solution}\n")
        write_print_to_log('\n\n--------------------子任务解决方案--------------------\n')
        write_print_to_log(f"Subtask solution:\n {subtask_solution}")

# 数据保存，保存任务解决方案到pickle
import pickle
with open('task2_solution_list.pkl', 'wb') as f:
    pickle.dump(task_solution_list, f)




--------------------开始处理任务 1: 收集中国智能服务机器人产业相关数据，包括协会年报、企业财报、市场规模等   --------------------

最相关的数据块编号: ['3', '13', '15', '19', '21', '30']


--------------------子任务解决方案--------------------

Subtask solution:
 ## 中国智能服务机器人产业数据综合分析报告

### 一、核心产业链数据概览

#### 1. 市场规模与技术渗透趋势
根据行业研究报告综合分析，中国智能服务机器人产业正处于高速成长期。在AI大模型技术驱动下，**2024年消费级机器人市场（含智能眼镜、家用设备等）规模突破百亿级**，其中智能眼镜品类因Ray-Ban Meta等爆品带动，出货量增速显著提升。工业及专业服务机器人领域，高阶自动驾驶（L3+）的商业化进程为物理AI场景落地提供重要参照，2025年有望成为服务机器人规模化应用的关键拐点。

#### 2. 核心部件国产化进展
智能机器人依赖的核心硬件国产替代加速，尤以传感器与AI芯片表现突出：

| **核心部件** | **国产化里程碑**                          | **技术对标**       | **市场影响**                     |
|--------------|-------------------------------------------|--------------------|----------------------------------|
| CIS传感器    | 思特威SC5A0CS（1英寸大底，22nm工艺）      | 索尼LYT-900        | 华为Pura 80 Ultra采用国产主摄   |
| 车载SoC      | 地平线征程系列、黑芝麻A1000               | 英伟达Orin         | 搭载于20余款国产车型             |
| AI端侧芯片   | 瑞芯微RK3588、华为昇腾                    | 高通骁龙AR1        | 雷鸟X3 Pro等AR眼镜采用国产平

In [27]:
# 把任务解决方案写入到报告中，形成一篇内容翔实的报告

auto_organize_sections_prompt_template = """
你是一个专业的金融研究报告编辑，需要将以下多个子话题的内容归类到标准的金融研究报告结构中。

每个子话题如下：
-----
{task_content_pairs}
-----

请将这些子任务内容归类到以下标准章节中：
1. 行业概况与现状
2. 行业结构与竞争格局
3. 政策与技术驱动因素
4. 行业前景与情景模拟
5. 进入与退出策略建议

请输出一个结构化结果，格式如下：
#1# 子任务标题
归类章节：行业概况与现状
内容摘要：该部分内容主要介绍了智能风控与大数据征信服务的基本定义和当前发展情况。

#2# 子任务标题
归类章节：行业前景与情景模拟
内容摘要：该部分内容主要构建了行业未来三年的增长模型与情景预测。

...

要求：
- 不要遗漏任何子任务；
- 不要添加解释或说明；
- 严格按照输出格式；
- 章节名称必须从上面5个中选择；
- 内容摘要要简明扼要。
"""

# 生成子任务标题 + 内容的拼接字符串
task_content_pairs_str = ""
for idx, (task, content) in enumerate(zip(task_list, task_solution_list), 1):
    task_content_pairs_str += f"#{idx}# {task}\n"

# 构建 Prompt
organize_prompt = auto_organize_sections_prompt_template.format(
    task_content_pairs=task_content_pairs_str.strip()
)

# 调用模型
response = deepseek_v3.invoke(organize_prompt)
sleep(1)
organize_result = response.content.strip()


In [28]:
from collections import defaultdict
import re

# 正则提取归类结果
pattern = r'#(\d+)# (.*?)\n归类章节：(.*?)\n内容摘要：(.*?)(?=\n#|\Z)'
matches = re.findall(pattern, organize_result, re.DOTALL)

# 标准章节列表
standard_sections = [
    "行业概况与现状",
    "行业结构与竞争格局",
    "政策与技术驱动因素",
    "行业前景与情景模拟",
    "进入与退出策略建议",
]

# 构建结构化归类字典
section_content_map = defaultdict(list)

# matches 是你提供的模型输出结果
for idx, task_title, model_section, summary in matches:
    # 清洗模型输出的章节名（去除前后空格/换行）
    cleaned_section = model_section.strip()

    # 初始化匹配章节为 None
    matched_section = None

    # 模糊匹配标准章节
    for std_sec in standard_sections:
        if std_sec in cleaned_section or cleaned_section in std_sec:
            matched_section = std_sec
            break

    # 如果完全没匹配上，归类为 "其他分析"
    if not matched_section:
        matched_section = "其他分析"

    # 清洗任务标题（去除前后空格）
    cleaned_task_title = task_title.strip()

    # 找到对应的完整内容
    full_content = None
    for task, content in zip(task_list, task_solution_list):
        if cleaned_task_title in task or task in cleaned_task_title:
            full_content = content.strip()
            break

    if full_content:
        section_content_map[matched_section].append(idx)

section_content_map



defaultdict(list,
            {'行业概况与现状': ['1', '2', '9', '10'],
             '行业结构与竞争格局': ['3', '4'],
             '政策与技术驱动因素': ['5'],
             '行业前景与情景模拟': ['6', '8'],
             '进入与退出策略建议': ['7']})

In [29]:
full_report = f"# {industry_input} 行业金融研究报告\n\n"

# 添加摘要
# full_report += "## 摘要\n\n"
# full_report += (
#     "本报告围绕智能风控与大数据征信服务行业展开系统研究，涵盖行业发展现状、市场规模、产业链结构、"
#     "竞争格局、政策环境与技术驱动因素等多个维度。通过分析行业协会年报、企业财报及相关政策文件，"
#     "评估行业生命周期阶段与未来发展趋势，并构建未来行业增长模型及情景模拟。报告同时提供行业进入与退出策略建议，旨在为投资者与行业参与者提供决策支持。"
# )
seen_content = set()

# 按照标准章节顺序拼接
for section_title in standard_sections:
    idx = section_content_map.get(section_title)
    # 获取对应章节的内容
    contents = []
    if idx:
        for num in idx:
            # 注意这里的i是字符串，需要转换为整数
            task_index = int(num) - 1  # 索引从0开始
            if task_index < len(task_solution_list):
                content = task_solution_list[task_index].strip()
                if content:  # 如果内容不为空
                    contents.append(content)
    if contents:
        full_report += f"## {section_title}\n\n"
        for content in contents:
            clean_content = re.sub(r'\s+', ' ', content).strip()
            if clean_content not in seen_content:
                full_report += content + "\n\n"
                seen_content.add(clean_content)

# 保存为 Markdown 文件
with open("field_financial_report_final.md", "w", encoding="utf-8") as f:
    f.write(full_report)

print("✅ field_financial_report_final.md")

✅ field_financial_report_final.md


In [30]:
from time import sleep
# 对每个章节的内容进行润色，一方面是围绕主题来写，纠正幻觉
polish_section_prompt_template = """
你是一个专业的金融研究报告编辑，负责对本章节内容进行润色与整合，目标是输出一个**逻辑清晰、语言专业、无幻觉**的完整章节内容，以markdown文本格式输出。

【报告主题】
-----
{query}
-----

【完整报告目录结构】
-----
{report_structure}
-----

【当前章节标题】
-----
{section_title}
-----

【当前章节的原始内容】
-----
{section_content}
-----

【润色要求】
1. **围绕当前章节标题**组织内容，确保内容紧扣主题；
2. **整理好排版，输出内容应包含当前章节标题**
3. **去掉原始内容中多余的空格、换行符和格式混乱的部分**；
4. **对于原始内容中的表格**，如果有并且是完整的，请使用Markdown格式输出，确保表格内容清晰、易读；
5. **理清本章节在整体报告中的逻辑定位**，整合原始内容，删除冗余信息，合并重复内容，不遗漏重要信息和表述；
6. **确保语言正式、专业**，符合金融研究报告的写作规范；
7. **纠正可能存在的数据错误、逻辑矛盾或幻觉内容**；
8. **仅基于原始内容进行润色，不添加任何原始内容中没有的信息或数据**；
9. **输出格式为完整的markdown格式的研究报告章节**，不要包含解释性语句（如“我将如何整合”）；

特别的，对于当前章节的标题和子标题、列表等，你需要尤其注意格式：
1. 当前章节标题{section_title}为一级标题（#）格式，例如"# 一、"和"# 二、"
2. 其他的子标题不允许使用"# 一、"和"# 二、"这样的中文序号标题格式，可以使用"(一)"或者"(二)"这样的格式，或者使用"1."、"2."这样的数字序号格式等等
2. 对于原始内容进行排版，如果原始内容里面出现了某个子标题内容特别长，需要对其进行浓缩符合子标题风格

请直接输出润色后的章节内容：
"""

section_idx_title = [
    "一、行业概况与现状",
    "二、行业结构与竞争格局",
    "三、政策与技术驱动因素",
    "四、行业前景与情景模拟",
    "五、进入与退出策略建议",
]

polished_result_list = []
# 按照标准章节顺序拼接
for i in range(len(standard_sections)):
    section_title = standard_sections[i]
    print(f"正在润色章节：{section_idx_title[i]}")
    idx = section_content_map.get(section_title)
    # 获取对应章节的内容
    contents = []
    if idx:
        for num in idx:
            # 注意这里的i是字符串，需要转换为整数
            task_index = int(num) - 1  # 索引从0开始
            if task_index < len(task_solution_list):
                content = task_solution_list[task_index].strip()
                if content:  # 如果内容不为空
                    contents.append(content)
    if contents:
        polish_section_prompt = polish_section_prompt_template.format(
            query=task2,  # 输入的研究任务
            report_structure="\n".join([f"{i+1}. {title}" for i, title in enumerate(standard_sections)]),  # 报告结构
            section_title=section_idx_title[i],  # 当前章节标题
            section_content='\n\n'.join(contents)  # 当前章节内容
        )
        # 调用模型进行润色
        polish_response = deepseek_r1.invoke(polish_section_prompt)
        sleep(2)
        polished_content = polish_response.content.strip()
        polished_result_list.append(polished_content)  # 存储润色后的内容

# pickle保存润色后的内容
import pickle
with open('field_polished_result_list.pkl', 'wb') as f:
    pickle.dump(polished_result_list, f)

正在润色章节：一、行业概况与现状
正在润色章节：二、行业结构与竞争格局
正在润色章节：三、政策与技术驱动因素
正在润色章节：四、行业前景与情景模拟
正在润色章节：五、进入与退出策略建议


In [31]:
import akshare as ak
import pandas as pd
from get_capital_structure import get_capital_structure_cn_ths, get_capital_structure_hk_ths
from get_cn_control import get_cn_control
from get_company_field_compare import get_cn_company_field_compare, get_hk_company_field_compare
from get_company_finance_summary import get_company_finance_summary_cn,get_company_finance_summary_hk
from get_company_intro import get_company_profile_ths_cn, get_company_profile_ths_hk, get_cn_company_profile_ak, get_hk_company_profile_ak
from get_financial_data_annual import download_cn_financial_data, download_hk_financial_data
from get_rating_info import get_hk_rating_info, get_cn_rating_info
from get_stock_info import get_cn_stock_info, get_hk_stock_info
from get_worth_predict import get_cn_worth_predict, get_hk_worth_predict
from datetime import datetime, timedelta
import time
import requests
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from typing import Dict, Any
from typing_extensions import Annotated, TypedDict
import json
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from typing import Dict, Optional, Literal
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
import ast
import pickle

from typing import List

from langchain_core.tools import InjectedToolArg, tool
from typing_extensions import Annotated
import time
from time import sleep
# 支持matplotlib输出中文
import matplotlib.pyplot as plt
import chineseize_matplotlib
plt.rcParams['axes.unicode_minus'] = False # 解决负号'-'显示为方块的问题

In [32]:
# 可以支持把llm生成结果保存为markdown的函数
import os
import pandas as pd

writer_data_value = {}
writer_data_desc = {}

class MarkdownReportBuilder:
    def __init__(self, report_title="行业研报", output_path="field_report.md", stream_out_path = "field_report_stream_output.md", image_dir="./field_img/"):
        self.title = report_title
        self.content = []
        self.current_content = ""  # 保存当前写了的临时的结果
        self.image_counter = 0
        self.table_couneter = 0
        self.image_dir = image_dir
        self.output_path = output_path
        self.stream_out_path = stream_out_path
        
        # 创建图片目录（如果不存在）
        if not os.path.exists(self.image_dir):
            os.makedirs(self.image_dir)
    # 实时输出到文件    
    def stream_output(self, text):
        """实时输出到文件"""
        with open(self.stream_out_path, "a", encoding="utf-8") as f:
            f.write(text + "\n")

    def add_paragraph(self, text):
        """添加一段文字"""
        self.content.append(f"{text}\n")
        self.current_content += f"{text}\n"
        self.stream_output(f"{text}\n")

    def add_table(self, df: pd.DataFrame, caption=None):
        """添加一个 DataFrame 表格"""
        if caption:
            self.content.append(f"**{caption}**\n")
            self.current_content += f"**{caption}**\n"
            self.stream_output(f"**{caption}**\n")
            
        self.content.append(df.to_markdown(index=False) + "\n")
        self.current_content += df.to_markdown(index=False) + "\n"
        self.stream_output(df.to_markdown(index=False) + "\n")
        self.table_couneter += 1

    def add_image(self, image_path, caption=""):
        """添加一张图片（将原图复制到指定目录）"""
        import shutil
        new_image_name = f"image_{self.image_counter}{os.path.splitext(image_path)[-1]}"
        new_image_path = os.path.join(self.image_dir, new_image_name)
        shutil.copyfile(image_path, new_image_path)
        self.content.append(f"![{caption}]({new_image_path})\n")
        self.current_content += f"![{caption}]({new_image_path})\n"
        self.stream_output(f"![{caption}]({new_image_path})\n")
        self.image_counter += 1

    def save(self):
        """保存报告为 Markdown 文件"""
        with open(self.output_path, "w", encoding="utf-8") as f:
            f.write(f"---\ntitle: 行业研报-{industry_input}\n---\n\n")
            f.write("\n".join(self.content))
        print(f"报告已保存至：{self.output_path}")

        
    def clear_current_content(self):
        """清空当前内容"""
        self.current_content = ""
    
    def get_current_content(self):
        """获取当前内容"""
        return self.current_content


# 全局报告对象（适用于单次任务）
report_builder = MarkdownReportBuilder(report_title="行业研报"+ " - " + industry_input, output_path="field_report.md", image_dir="./field_img/")

@tool
def add_plain_text(content: str):
    """
    添加一段文字内容到报告中，除了图片和表格，其他内容都必须调用这个函数才能添加到报告中。
    """
    report_builder.add_paragraph(content)
    return "已添加该段文字"

@tool
def add_table(df_key_name: str, caption: str = None):
    """把调用plain_text_dataframe_to_table函数生成的DataFrame添加到报告中
    Args:
        df_key_name (str): writer_data_value里面指向DataFrame的键名
        caption (str): 表格的编号
    """
    if df_key_name not in writer_data_value:
        return f"没有找到键名为 '{df_key_name}' 的DataFrame，请先调用plain_text_dataframe_to_table函数生成表格数据。检查表格数据是否成功生成保存！"
    df = writer_data_value[df_key_name]
    report_builder.add_table(df,caption)
    table_idx = report_builder.table_couneter
    return "已添加该表格，该表的编号为 表{}".format(table_idx)

@tool
def add_image(image_path: str, caption: str = None):
    """把调用draw_data_date_one_list_line_plot等画图函数生成的图片添加到报告中
    Args:
        image_path (str): 已经画好图的图片的存储路径
        caption (str): 图片的标题或说明文字
    """
    report_builder.add_image(image_path, caption)
    img_idx = report_builder.image_counter
    return "已添加该图片，该图片的编号为 图片{}".format(img_idx)

@tool
def save_report(output_path: str = "field_report.md"):
    """保存报告为markdown文件"""
    report_builder.save()
    return f"报告已保存至文件： {output_path}"

# 输入为dict的形式，变成一个dataframe，存在collected_data_value中，键名为df_key_name
# 例如{
# "H":['a','b','c,'d']
# "a":[1,2,3,4],
# "b":[5,6,7,8],}
@tool
def plain_text_dataframe_to_table(
    df_key_name: str,
    data_dict: dict
):
    """
    将报告中文本格式的表格转化为dataframe，输入需要把表格的数据以字典形式传入，最后会把数据保存到writer_data_value中，保存的key为指定的参数df_key_name，供后续调用add_table函数插入到报告中。
    Args:
        df_key_name (str): 保存DataFrame的键名，不要和已有的键名冲突，可以取为插入文档中表格的名字
        data_dict (dict): 包含数据的字典，键为列名，值为列数据列表

    Returns:
        str: 返回保存成功的信息
    """
    global writer_data_value, writer_data_desc, report_builder
    try:
        df = pd.DataFrame(data_dict)
    except Exception as e:
        return f"数据转换为DataFrame失败: {str(e)}，请注意数据格式是否正确"
    writer_data_value[df_key_name] = df
    writer_data_desc[df_key_name] = f"写作时需要使用的数据表，键名为{df_key_name}"
    return f"DataFrame已保存到writer_data_value中，键名 '{df_key_name}' 中，稍后可以通过该键名调取插入表格函数add_table给报告中添加表格，该表格的编号为 表{report_builder.table_couneter + 1}。"

# # 画图函数
# 支持输入数据列表y_list，日期列表date_list，画出来，保存为图片，返回图片保存路径
@tool
def draw_data_date_one_list_line_plot(
    y_list, 
    date_list,
    title="数据变化趋势", 
    xlabel="日期", 
    ylabel="数值", 
    legend_label="数据", 
    image_path="data_trend.jpg") -> str:
    """
    绘制一维简单数据变化趋势折线图，支持输入单个的数据列表和日期列表，要求数据列表和日期列表长度一致，按照日期顺序排序
    Args:
        y_list (List[float]): 数据列表
        date_list (List[str]): 日期列表，格式为"YYYY-MM-DD"
        title (str): 图表标题
        xlabel (str): x轴标签
        ylabel (str): y轴标签
        legend_label (str): 图例标签
    returns:
        str: 图片保存路径
    """
    from matplotlib import pyplot as plt
    import os

    # 检查输入数据长度是否一致
    if len(y_list) != len(date_list):
        return ("数据列表和日期列表长度不一致")

    # 转换日期字符串为日期对象
    try:
        date_list = [datetime.strptime(date, "%Y-%m-%d") for date in date_list]
    except ValueError:
        return ("日期格式错误，请使用YYYY-MM-DD格式的日期字符串")

    # 绘图
    plt.figure(figsize=(10, 5))
    plt.plot(date_list, y_list, marker='o', label=legend_label)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45)
    plt.legend()
    
    # 保存图片
    # image_path = "data_trend.jpg"
    plt.tight_layout()
    plt.savefig(image_path)
    plt.close()

    return "图片保存路径为: " + image_path + "，稍后可以通过调用add_image函数将图片插入到报告中"

# 绘制一维的数据占比图
@tool
def draw_data_one_list_pie_chart(y_list, labels, title="数据占比图", image_path="data_pie_chart.jpg") -> str:
    """
    绘制一维数据占比饼图，支持输入单个的数据列表和标签列表，要求数据列表和标签列表长度一致
    Args:
        y_list (List[float]): 数据列表，饼图的每一块对应一个数据值，需要保证所有数据非负
        labels (List[str]): 标签列表
        title (str): 图表标题
        image_path (str): 图片保存路径
    """
    from matplotlib import pyplot as plt

    # 检查输入数据长度是否一致
    if len(y_list) != len(labels):
        return ("数据列表和标签列表长度不一致")
    # 检查数据是否为非负数
    if any(value < 0 for value in y_list):
        return ("数据列表中的值必须为非负数，请检查数据输入，该数据不适合用饼图展示，请尝试使用条形图draw_data_one_list_bar_chart或者折线图draw_data_date_one_list_line_plot")

    # 绘图
    plt.figure(figsize=(8, 8))
    plt.pie(y_list, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title(title)
    
    # 保存图片
    plt.tight_layout()
    plt.savefig(image_path)
    plt.close()

    return "图片保存路径为: " + image_path + "，稍后可以通过调用add_image函数将图片插入到报告中"

# 绘制一维数据的条形图，例如用于展示行业地位，要求能支持将其中某一个柱子的颜色突出
@tool
def draw_data_one_list_bar_chart(y_list, labels, highlight_index=None, title="数据条形图", xlabel="类别", ylabel="数值", image_path="data_bar_chart.jpg") -> str:
    """
    绘制一维数据条形图，支持输入单个的数据列表和标签列表，要求数据列表和标签列表长度一致
    Args:
        y_list (List[float]): 数据列表
        labels (List[str]): 标签列表
        highlight_index (Optional[int]): 突出显示的柱子索引，默认为None表示不突出显示
        title (str): 图表标题
        xlabel (str): x轴标签
        ylabel (str): y轴标签
        image_path (str): 图片保存路径
    """
    from matplotlib import pyplot as plt

    # 检查输入数据长度是否一致
    if len(y_list) != len(labels):
        raise ValueError("数据列表和标签列表长度不一致")

    # 绘图
    plt.figure(figsize=(10, 5))
    bars = plt.bar(labels, y_list, color='blue')

    # 突出显示指定柱子
    if highlight_index is not None and 0 <= highlight_index < len(bars):
        bars[highlight_index].set_color('red')

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    # 保存图片
    plt.tight_layout()
    plt.savefig(image_path)
    plt.close()

    return "图片保存路径为: " + image_path + "，稍后可以通过调用add_image函数将图片插入到报告中"


markdown_tools = [
    add_plain_text,
    add_table,
    add_image,
    save_report,
    plain_text_dataframe_to_table,
    draw_data_date_one_list_line_plot,
    draw_data_one_list_pie_chart,
    draw_data_one_list_bar_chart
]

markdown_agent = tool_agent.bind_tools(markdown_tools)
markdown_format_prompt_template = """
你是一个专业的金融研究报告编辑助手，负责将原始文稿内容结构化地写入 Markdown 格式的调研报告中。

你的任务是：
1. 分析【章节内容】，识别出需要结构化处理的内容（如表格、图表、普通文本等）；
2. 按照以下规则调用工具函数，将内容写入报告；
3. 最终使用 `save_report` 保存报告。

【调用工具的规则】：

- **遇到表格内容**（如：`|列1|列2|\n|---|---|`）时：
  1. 使用 `plain_text_dataframe_to_table` 将其转换为 DataFrame；
  2. 使用 `add_table` 将表格插入报告；
  3. 在输出内容中不再保留原始文本格式表格。

- **遇到图表描述**（如“收入趋势”、“行业占比”、“柱状图”等）时：
  1. 使用对应的绘图工具（如 `draw_data_date_one_list_line_plot` 或 `draw_data_one_list_pie_chart`）生成图片；
  2. 使用 `add_image` 将图片插入报告；

- **遇到普通文本内容**（非表格、非图表）时：
  1. 使用 `add_plain_text` 添加内容，保证内容中没有转义字符或unicode值；
  2. 保持段落结构清晰，不遗漏关键信息。

- **完成内容输出后**：
  1. 使用 `save_report` 保存最终的 Markdown 报告。

【注意事项】：
- 你的下游就是研究报告，千万一定不要对中文进行转义，add_plain_text的参数不要出现转义字符或者unicode值；
- 你不能对原始内容进行任何润色、改写、缩写或扩展；
- 你不能添加任何你自己的解释、评价、分析、总结或者你的思维过程；
- 你不能修改原文稿中的任何数据、格式或语序，除非是识别到文本格式的表格需要调用表格处理函数输出，不保留原始的文本形式表格内容；
- 所有文本内容必须原样通过 `add_plain_text` 添加到报告中；
- 表格内容必须先调用 `plain_text_dataframe_to_table` 再调用 `add_table`添加;
- 要增加图表，必须先调用绘图函数生成图片，再调用 `add_image` 添加到报告中。

原始文稿的章节标题为：
-----
{section_title}
-----

原始文稿的章节内容为：
-----
{section_content}
-----

请开始调用工具，将内容结构化写入报告。
"""

In [33]:
# for polist list
for i in range(len(polished_result_list)):
    polished_content = polished_result_list[i]
    section_title = section_idx_title[i]  # 当前章节标题
    print(f"\n\n--------------------开始处理章节 {i+1}: {section_title} --------------------\n")
    writer_history_message = []
    while True:
        # 构建章节写作提示
        markdown_format_prompt = markdown_format_prompt_template.format(
            section_title=section_title,  # 当前章节标题
            section_content=polished_content  # 当前章节内容
        )
        # 调用markdown写作助手
        writer_history_message.append(HumanMessage(
            content=markdown_format_prompt  # 当前章节内容
        ))
        markdown_writer_output_response = markdown_agent.invoke(writer_history_message,enable_thinking=False) # 输出到markdown中
        sleep(1)  # 等待1秒，避免过快调用
        writer_history_message.append(markdown_writer_output_response)
        if markdown_writer_output_response.tool_calls is None or len(markdown_writer_output_response.tool_calls) == 0:
            break
        # 执行markdown输出工具
        for tool_call in markdown_writer_output_response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']
            selected_tool = globals()[tool_call["name"]]
            current_data = selected_tool.invoke(tool_args)  # 正确调用方式
            if tool_name == "save_report":
                break
            writer_history_message.append(ToolMessage(
                        content=str(current_data),  # 转为字符串
                        name=tool_name,  # 工具名称
                        tool_call_id=tool_call['id']  # 匹配对应的 tool_call ID
            ))
            write_print_to_log(f"调用工具 {tool_name}, 参数: {tool_args}, 返回结果: {current_data}")
            with open(trace_log, 'a', encoding='utf-8') as f:
                f.write(f"调用工具 {tool_name}, 参数: {tool_args}, 返回结果: {current_data}\n")
        if tool_name == "save_report":
                break
        
# 手动调用save_report
report_builder.save()




--------------------开始处理章节 1: 一、行业概况与现状 --------------------

调用工具 add_plain_text, 参数: {'content': '# 一、行业概况与现状'}, 返回结果: 已添加该段文字
调用工具 add_plain_text, 参数: {'content': '### (一) 市场规模与增长趋势\n中国智能服务机器人产业正处于高速成长期，在AI大模型技术驱动下，市场规模持续扩张。2024年消费级机器人市场（含智能眼镜、家用设备等）规模突破百亿级，其中智能眼镜品类因爆品带动出货量增速显著提升。工业及专业服务机器人领域，高阶自动驾驶（L3+）的商业化进程为物理AI场景落地提供参照，2025年有望成为服务机器人规模化应用的关键拐点。具体表现为：\n- **消费端**：2025年Q1全球AI智能眼镜销量达60万台，同比激增216%；洛图科技预测2025年全球出货量将达376万副，同比大增139%。\n- **工业端**：高阶智驾（L3及以上）进入规模化落地前夜，L2+功能在10万元级车型快速普及（如比亚迪策略），L3级乘用车落地在即，L4级商用车试点扩大。\n- **硬件需求**：全球云服务资本支出2023年达2480亿美元，2025年预期增至3270亿美元（Marvell数据），驱动端侧AI硬件需求扩张。'}, 返回结果: 已添加该段文字
调用工具 plain_text_dataframe_to_table, 参数: {'df_key_name': '核心部件国产化进展', 'data_dict': {'核心部件': ['CIS传感器', '车载SoC', 'AI端侧芯片'], '国产化里程碑': ['思特威SC5A0CS（1英寸大底，22nm工艺）', '地平线征程系列、黑芝麻A1000', '瑞芯微RK3588、华为昇腾'], '技术对标': ['索尼LYT-900', '英伟达Orin', '高通骁龙AR1'], '市场影响': ['华为Pura 80 Ultra采用国产主摄', '搭载于20余款国产车型', '雷鸟X3 Pro等AR眼镜采用国产平台']}}, 返回结果: DataFrame已保存到writer_data_value中，键名 '核心部件国产化进展' 中，稍后可以通过该键名调取

In [34]:
import pypandoc
import os

def convert_markdown_to_docx(input_file, output_file):
    """
    使用 Pandoc 将 Markdown 文件转换为 DOCX 格式
    自动处理表格、格式和样式
    """
    try:
        # 检查文件是否存在
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"输入文件 {input_file} 不存在")

        # 使用 Pandoc 进行转换
        output = pypandoc.convert_file(
            input_file,
            'docx',
            outputfile=output_file,
            extra_args=[
                '--columns=80',
            ]
        )


    except Exception as e:
        print(f"转换过程中发生错误：{str(e)}")


input_md = "field_report.md"
output_docx = "Industry_Research_Report.docx"
    
# 执行转换
convert_markdown_to_docx(input_md, output_docx)